In [ ]:
import numpy as np
import math
import random
import time

In [ ]:
# State class to store info about a state
class State:
    def __init__(self, state, depth, parent = None):
        self.state = state
        self.x, self.y = np.where(state == 0)
        self.depth = depth
        self.parent = parent

    def set_energy(self, heuristic_fn):
        self.energy = heuristic_fn(self.state)

In [ ]:
def displaced_heuristic(state):
    d = 0
    for i in range(1, 9):
        i_x, i_y = np.where(state == i)
        f_x, f_y = np.where(np.array(goal) == i)
        d += (i_x[0] != f_x[0] or i_y[0] != f_y[0])
    return d

def manhattan_heuristic(state):
    d = 0
    for i in range(1, 9):
        i_x, i_y = np.where(state == i)
        f_x, f_y = np.where(np.array(goal) == i)
        d += abs(i_x[0] - f_x[0]) + abs(i_y[0] - f_y[0])
    return d

def new_heuristic(state):
    return displaced_heuristic(state) * manhattan_heuristic(state)

heuristics = [displaced_heuristic, manhattan_heuristic, new_heuristic]

In [ ]:
def random_neighbour(state):
    neighbours = []
    x, y = state.x, state.y
    moves = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    for dx, dy in moves:
        if 0 <= (x + dx) <= 2 and 0 <= (y + dy) <= 2:
            new_state = state.state.copy()
            new_state[x, y], new_state[x + dx, y + dy] = new_state[x + dx, y + dy], new_state[x, y]
            neighbours.append(new_state)
    return State(neighbours[random.randint(0, len(neighbours) - 1)], state.depth + 1, state)

In [ ]:
def boltzman_prob(delta, temp):
    return math.e**(-delta / temp / 100)

In [ ]:
def alpha_cooling(temp):
    return temp * alpha

In [ ]:
def simulated_annealing(init, n_iterations, temp, heuristic_fn, generator_fn, prob_fn, cooling_fn, min_temp):
    mini = init
    mini.set_energy(heuristic_fn)
    n_nodes = 0
    random_walk = False
    while temp >= min_temp and mini.energy > 0:
        curr_state = mini
        for i in range(n_iterations):
            new_state = random_neighbour(curr_state)
            new_state.set_energy(heuristic_fn)
            n_nodes = n_nodes + 1
            if new_state.energy < mini.energy:
                mini = new_state
            del_e = new_state.energy - curr_state.energy
            if del_e < 0 or random.random() < prob_fn(del_e, temp):
                curr_state = new_state
                random_walk = True
        temp = cooling_fn(temp)
    return {"status": mini.energy == 0, "state": mini, "n_nodes": n_nodes, "random_walk": random_walk}

In [ ]:
# Helper Function to print path from start to goal
def print_path(goal):
    s = ""
    path = [goal]
    while not (path[0].state == init).all():
        path.insert(0, path[0].parent)
    s = s + f"\tPath (Cost = {len(path) - 1}):- \n\n"
    for i in path:
        s = s + str(i.state) + "\n\n"
    return s

In [ ]:
# Initial State
init = np.loadtxt("input.txt", dtype=int)
start = State(init, 0)

# Required Goal State
goal = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

n_iterations = 10
start_temp = 1 # Tmax
end_temp = 0.0001 # Tmin
alpha = 0.8 # Tnew = T * alpha
heuristic_choice = 1 # Manhattan

start_time = time.time()
final = simulated_annealing(start, n_iterations, start_temp, heuristics[heuristic_choice], random_neighbour, boltzman_prob, alpha_cooling, end_temp)
end_time = time.time()

In [ ]:
with open("output.txt", "w") as f:
    if final["status"]:
        f.write("Goal State was reached\n\n")
    else:
        f.write("Goal was not found\n\n")
    match heuristic_choice:
        case 0:
            f.write("H1 (Number of Displaced Tiles) was used as the Heuristic Function\n")
        case 1:
            f.write("H2 (Manhattan Distance of Each Tile) was used as the Heuristic Function\n")
        case 2:
            f.write("H3 = H1 * H2 was used as the Heuristic Function\n")
    f.write(f"Start Temperature was taken as {start_temp} and End Temperature was taken as {end_temp}\n")
    f.write("Alpha Cooling Function was chosen: T_new = T * alpha (alpha < 1)\n")
    f.write(f"Start state was \n{init}\nGoal State was \n{goal}\n\n")
    f.write(print_path(final["state"]))
    f.write("Total Number of Nodes explored: " + str(final["n_nodes"]) + "\n")
    f.write("Total Time Taken: {:.2f}ms\n".format((end_time - start_time) * 1000))
    if final["random_walk"]:
        f.write("The Algorithm went into a Random Walk\n")
    else:
        f.write("The Algorithm did not go in a Random Walk\n")

In [ ]:
with open("output.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        print(line, end = "")

Goal State was reached

H2 (Manhattan Distance of Each Tile) was used as the Heuristic Function
Start Temperature was taken as 1 and End Temperature was taken as 0.0001
Alpha Cooling Function was chosen: T_new = T * alpha (alpha < 1)
Start state was 
[[1 2 3]
 [4 6 0]
 [7 5 8]]
Goal State was 
[[1 2 3]
 [4 5 6]
 [7 8 0]]

	Path (Cost = 3):- 

[[1 2 3]
 [4 6 0]
 [7 5 8]]

[[1 2 3]
 [4 0 6]
 [7 5 8]]

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

Total Number of Nodes explored: 10
Total Time Taken: 3.47ms
The Algorithm went into a Random Walk
